# Project: SVD using Fixed Point Iterations

## Group members
* Jake Swartwout
* Lara Chunko
* Yunhan Yang

The link to the repository: https://github.com/j2kun/svd/blob/master/svd.py

At a high level, this software performs the "Singular Value Decomposition" (SVD) of a matrix. This means that it is splitting any matrix A (it doesn't even need to be square) into 3 separate matrices. We represent it like this:

$$
A = U \Sigma V^T
$$

With the following properties:

$U$ and $V$ have orthonormal columns

$\Sigma$ is a diagonal matrix

This may seem familiar to diagonalization, as it does something similar. One of the main benefits however, is that our matrix A does not have to be square for SVD

- accuracy for size of matrix
- accuracy for different random values
- convergence tests
- Lara: runtime vs standard software (linalg.svd?)
- compare multiple svd packages?
- Jake: break it (have two equal eigenvalues?)

Multiply U by UT to see if it's the identity?

In [36]:
from svdMaster.svd import svd as testSVD
from numpy.linalg import svd as npSVD
from scipy.linalg import svd as scipySVD

import numpy as np

#create a timer
import timeit as t

In [39]:
#test array
A = np.random.rand(10,10) 

#applying testSVD
start_t = t.timeit()
test_sigma, test_U, test_VT = testSVD(A)
end_t = t.timeit()
test_time = end_t - start_t
print("testSVD runtime: ", test_time)
test_recombined = test_U @ np.diag(test_sigma) @ test_VT

#applying scipySVD 
start_t = t.timeit()
scipy_U, scipy_sigma, scipy_VT = scipySVD(A) 
end_t = t.timeit()
scipy_time = end_t - start_t
print("scipySVD runtime: ", scipy_time)
scipy_recombined = scipy_U @ np.diag(scipy_sigma) @ scipy_VT

#applying npSVD
start_t = t.timeit()
np_U, np_sigma, np_VT = npSVD(A)
end_t = t.timeit()
np_time = end_t - start_t
print("npSVD runtime: ", scipy_time)
np_recombined = np_U @ np.diag(np_sigma) @ np_VT

converged in 6 iterations!
converged in 28 iterations!
converged in 21 iterations!
converged in 52 iterations!
converged in 41 iterations!
converged in 12 iterations!
converged in 18 iterations!
converged in 9 iterations!
converged in 5 iterations!
converged in 2 iterations!
testSVD runtime:  4.490499998155428e-05
scipySVD runtime:  -0.0007957709999573126
npSVD runtime:  -0.0007957709999573126


In [40]:
#testing testSVD versus scipySVD
#loop through and find the difference between the two versions
test_scipy_recombined_difference = 0
#column
for i in range(len(test_recombined[0])):
    #row
    for j in range(len(test_recombined)):
        #print("test ", test_recombined[i,j])
        #print("scipy ", scipy_recombined[i,j])
        test_scipy_recombined_difference += abs(test_recombined[i,j] - scipy_recombined[i,j])
        
print("test_scipy_recombined_difference = ", test_scipy_recombined_difference)

#testing testSVD versus npSVD
#loop through and find the difference between the two versions
test_np_recombined_difference = 0
#column
for i in range(len(test_recombined[0])):
    #row
    for j in range(len(test_recombined)):
        #print("test ", test_recombined[i,j])
        #print("np ", np_recombined[i,j])
        test_np_recombined_difference += abs(test_recombined[i,j] - np_recombined[i,j])
        
print("test_np_recombined_difference = ", test_np_recombined_difference)

#testing scipySVD versus npSVD
#loop through and find the difference between the two versions
scipy_np_recombined_difference = 0
#column
for i in range(len(scipy_recombined[0])):
    #row
    for j in range(len(scipy_recombined)):
        #print("scipy ", scipy_recombined[i,j])
        #print("np ", np_recombined[i,j])
        scipy_np_recombined_difference += abs(scipy_recombined[i,j] - np_recombined[i,j])

print("scipy_np_recombined_difference = ", scipy_np_recombined_difference)

test_scipy_recombined_difference =  3.110584706478292e-13
test_np_recombined_difference =  3.110584706478292e-13
scipy_np_recombined_difference =  0.0


In [73]:
def split_combine_test(matrix):
    sigma, u, v = testSVD(A)
    return u @ np.diag(sigma) @ v

## Breaking it

This method does not seem very robust, so we decided to look into a few ways to break it.

1. My first idea is to have two eigenvalues of the same value. This would mean that the program would not be able to choose one.

In [12]:
#generate a random matrix
A = np.random.rand(5, 5)
#get the length of the first and second columns
length1 = np.linalg.norm(A[:, 0])
length2 = np.linalg.norm(A[:, 1])
#scale them to be the same length
A[:, 0] *= length2
A[:, 1] *= length1
Aprime = split_combine_test(A)
np.linalg.norm(Aprime - A)

2.1359173327318836e-15

2. Another idea would be to have all of them be the same length
3. We could also test for a rotation
4. Or test a matrix where two columns are at a very similar angle
5. Or a matrix where two columns are just scaled versions of each other

---

The following prompts may be useful, but you don't have to use them.

## Introduction

This method of SVD seems both unstable and inefficient, so we seek to see how accurate it as compared to the standard methods.

Describe the objective of your study, citing prior work as appropriate (papers, websites, etc.).  There is no requirement on citation style, but please try to be consistent.

## Methods

## Results and interpretation

## Conclusions and open questions